In [0]:
dbutils.fs.mount(source="wasbs://casestudygroup5@casestudystorage.blob.core.windows.net/", mount_point="/mnt/bronze", extra_configs = {"fs.azure.account.key.casestudystorage.blob.core.windows.net": "MlAPBstJ5N6ky5Vwg7jjX2Tp4LsX37g84lpJzfYTmBnFqH722wQnGL/Zi/y3bGyQfyMh5iH+I4d1+AStzbmFNw=="})

# Data Pre Processing

In [0]:
%fs ls dbfs:/mnt/bronze/

In [0]:
csv_file_path = "dbfs:/mnt/bronze/raw/light-duty-vehicles-2023-08-07.csv"
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

In [0]:
display(df)

In [0]:
from pyspark.sql.functions import *

In [0]:
df = df.withColumn("Alternative Fuel Economy City", col("Alternative Fuel Economy City").cast("double")).fillna(0, subset=["Alternative Fuel Economy City"])

In [0]:
df = df.withColumn("Alternative Fuel Economy Highway", col("Alternative Fuel Economy Highway").cast("double")).fillna(0, subset=["Alternative Fuel Economy Highway"])

In [0]:
df.display()

In [0]:
df=df.where(col("Fuel Code").isNotNull())

In [0]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).display()

In [0]:
df.write.format("csv").mode("overwrite").option("header",True).option("path","dbfs:/mnt/bronze/STG").saveAsTable("lightdutyvehicles_stg")

In [0]:
df=df.drop("Manufacturer URL","Electric-Only Range", "PHEV Total Range", "PHEV Type","Notes","Drive Train", "Charging Rate Level 2 (kW)", "Charging Rate DC Fast (kW)", "Charging Speed Level 1 (miles added per hour of charging)", "Charging Speed Level 2 (miles added per hour of charging)", "Charging Speed DC Fast (miles added per hour of charging)", "Battery Voltage", "Battery Capacity Amp Hours", "Battery Capacity kWh", "Seating Capacity")

In [0]:
df.printSchema()

In [0]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).display()

In [0]:
df=df.where(col("Fuel Configuration ID").isNotNull())

In [0]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).display()

In [0]:
df = df.withColumn("Alternative Fuel Economy Combined", col("Alternative Fuel Economy Combined").cast("double")).fillna(0, subset=["Alternative Fuel Economy Combined"])

In [0]:
display(df)

In [0]:
df4.groupBy("Fuel Code").count().show()

In [0]:
df=df.replace("H2","HY", subset="Fuel Code")

In [0]:
df=df.replace("E85_GSLN","E85", subset="Fuel Code")

In [0]:
df.write.format("csv").mode("overwrite").option("header",True).option("path","dbfs:/mnt/bronze/curated").saveAsTable("lightdutyvehicles_curated")

In [0]:
dbutils.fs.unmount("/mnt/bronze")